In [114]:
from typing import Dict, Tuple, List

class Model:
    def __init__(self, name: str, model, **kwargs):
        self.name = name
        self.model = model
        self.kwargs = kwargs
        
    def __str__(self) -> str:
        return self.name
    
    def __repr__(self) -> str:
        return self.name
    
    def get_model(self) -> Tuple:
        return (self.model, self.kwargs)

In [52]:
from sklearn.tree import DecisionTreeClassifier

def get_decision_tree(name: str = 'DecisionTree', criterion: str = 'gini', max_depth: int = 50, 
                      random_state: int = 42, class_weight: str = 'balanced', **kwargs) -> Model:
    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        random_state=random_state,
        class_weight=class_weight
    )
    return Model(name, model, **kwargs)

Model.DecisionTree = get_decision_tree

In [53]:
Model.DecisionTree()

DecisionTree

In [54]:
from sklearn.ensemble import RandomForestClassifier
from multiprocessing import cpu_count


def get_random_forest(name: str = 'RandomForest', n_estimators: int = 500, criterion: str = 'gini', 
                      max_depth: int = 30, random_state: int = 42, 
                      class_weight: str = 'balanced', n_jobs: int = cpu_count, **kwargs):
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        random_state=random_state,
        class_weight=class_weight,
        n_jobs=n_jobs
    )
    return Model(name, model, **kwargs)

Model.RandomForest = get_random_forest

In [55]:
Model.RandomForest()

RandomForest

In [105]:
from tensorflow.keras.layers import Input, Flatten, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC

In [126]:
from tensorflow.keras.models import Sequential

def get_sequential(default_name: str = 'Sequential'):
    def get_layers(*layers):
        def get_model(input_shape: Tuple[int], name: str = None, optimizer: str = 'nadam', 
                      loss: str = 'binary_crossentropy', metrics: List = None,
                      epochs: int = 1000, batch_size: int = 1024, 
                      validation_split: float = 0.1, shuffle: bool = True, verbose: bool = True, 
                      callbacks: List = None, **kwargs):
            name = name or default_name
            input_layer = Input(shape=input_shape)
            output_layer = Dense(1, activation="sigmoid")
            model = Sequential((input_layer,) + (output_layer,), name)
            
            matrics = metrics or [
                "accuracy",
                AUC(curve="ROC", name="auroc"),
                AUC(curve="PR", name="auprc")
            ]
            model.compile(
                optimizer=optimizer,
                loss=loss,
                metrics=metrics
            )
            
            kwargs.update({
                'epochs': epochs,
                'batch_size': batch_size,
                'validation_split': validation_split,
                'shuffle': shuffle,
                'verbose': verbose,
                'callbacks': callbacks
            })
            print(model.summary())
            return Model(name, model, **kwargs)
        return get_model
    return get_layers

Model.Sequential = get_sequential()

In [127]:
Model.Sequential()((104,20))

Model: "Sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 104, 1)            21        
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
None


Sequential

In [128]:
Model.Perceptron = get_sequential('Perceptron')()

In [129]:
Model.Perceptron((104,))

Model: "Perceptron"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 1)                 105       
Total params: 105
Trainable params: 105
Non-trainable params: 0
_________________________________________________________________
None


Perceptron

In [130]:
Model.MLP = get_sequential('MLP')

In [131]:
from keras_tqdm import TQDMNotebookCallback as ktqdm


Model.MLP(
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dense(32, activation="relu")
)((104,), callbacks=[
        EarlyStopping(monitor="val_loss", mode="min", patience=50),
        ktqdm(leave_outer=False)
    ])

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 1)                 105       
Total params: 105
Trainable params: 105
Non-trainable params: 0
_________________________________________________________________
None


MLP

In [132]:
Model.FFNN = get_sequential('FFNN')